# Worksheet 08

Name: Jeetkumar Vijaywargi 
UID: U00204914

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [1]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[5.2446949454293845, 3.1323056942177874, 8.246042743508399, 5.3991566008498735, 5.134435613662363, 5.09225063745143, 5.1676174875855985, 5.223705980092602, 5.104608095902072, 4.975234681986382]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [2]:
s2 = np.random.normal(8, 1, 10).tolist()
print(s2)

[9.216722633537195, 7.370041691731539, 7.889375829717049, 6.920389165382842, 8.368421291678374, 8.811836738133685, 6.9865623156462915, 9.096190547125564, 6.468840475968554, 7.097982177557282]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [3]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1.pop()
        data.append(p1)
    else:
        p2 = s2.pop()
        data.append(p2)
print(data)

[7.097982177557282, 4.975234681986382, 5.104608095902072, 5.223705980092602, 6.468840475968554, 5.1676174875855985, 5.09225063745143, 5.134435613662363, 5.3991566008498735, 8.246042743508399]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

The probability is half. There are parameter of other individual gaussian methods we used to generate this data. Other than the fair coin. 

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [4]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [ len(s1) / (len(s1) + len(s2)) , len(s2) / (len(s1) + len(s2)) ]
mean = [ sum(s1)/len(s1) , sum(s2)/len(s2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , sum(map(lambda x : (x - mean[1])**2, s2)) / len(s2) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))


[4.975234681986382, 5.104608095902072, 5.223705980092602, 5.1676174875855985, 5.09225063745143, 5.134435613662363, 5.3991566008498735]
[7.097982177557282, 6.468840475968554, 8.246042743508399]
P(S_1) = 0.7,  P(S_2) = 0.3
mean_1 = 5.156715585361474,  mean_2 = 7.270955132344746
var_1 = 0.014812406549277414,  var_2 = 0.5413678048354141


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [5]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x_i = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_0 | X_i) = P(X_i | S_0)P(S_0) / P(X_i)
    prob_s0_x_i = (pdf_i[0] * prob_s[0]) / prob_x_i
    prob_s0_x.append( prob_s0_x_i )
    # P(S_1 | X_i) = P(X_i | S_1)P(S_1) / P(X_i)
    prob_s1_x_i = (pdf_i[1] * prob_s[1]) / prob_x_i
    prob_s1_x.append( prob_s1_x_i )

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_0 = " + str(p[1]))
    print("Probability of coming from S_1 = " + str(p[2]))
    print()


point =  7.097982177557282
probability of observing that point if it came from cluster 0 =  0.0
probability of observing that point if it came from cluster 1 =  0.7002444472882792
point =  4.975234681986382
probability of observing that point if it came from cluster 0 =  6.827120509552089e-32
probability of observing that point if it came from cluster 1 =  9.173596202711429e-05
point =  5.104608095902072
probability of observing that point if it came from cluster 0 =  0.05534310854837133
probability of observing that point if it came from cluster 1 =  0.00024561270941308565
point =  5.223705980092602
probability of observing that point if it came from cluster 0 =  0.0009745408055254132
probability of observing that point if it came from cluster 1 =  0.0005781831405590303
point =  6.468840475968554
probability of observing that point if it came from cluster 0 =  0.0
probability of observing that point if it came from cluster 1 =  0.2458785790226317
point =  5.1676174875855985
probabilit

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [6]:
prob_c = [sum(prob_s0_x)/ len(prob_s0_x), sum(prob_s1_x) / len(prob_s1_x)]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x)  ]
var = [sum([x[0] * ((x[1] - mean[0])**2) for x in zip(prob_s0_x, data)]) / sum(prob_s0_x) , sum([x[0] * ((x[1] - mean[1])**2) for x in zip(prob_s1_x, data)]) / sum(prob_s1_x)]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.7,  P(S_2) = 0.3
mean_1 = 5.1416473388939075,  mean_2 = 6.378756701382762
var_1 = 0.002061920387540103,  var_2 = 1.3885691692101556


g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [7]:
prob_s0_x = []  # P(S_0 | X_i)
prob_s1_x = []  # P(S_1 | X_i)

for p in data:
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))

    # P(S_0 | X_i) = P(X_i | S_0)P(S_0) / P(X_i)
    prob_s0_x_i = (pdf_i[0] * prob_c[0]) / (pdf_i[0] * prob_c[0] + pdf_i[1] * prob_c[1])
    prob_s0_x.append(prob_s0_x_i)

    # P(S_1 | X_i) = P(X_i | S_1)P(S_1) / P(X_i)
    prob_s1_x_i = (pdf_i[1] * prob_c[1]) / (pdf_i[0] * prob_c[0] + pdf_i[1] * prob_c[1])
    prob_s1_x.append(prob_s1_x_i)

# Print updated posterior probabilities
probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_0 = " + str(p[1]))
    print("Probability of coming from S_1 = " + str(p[2]))
    print()

7.097982177557282
Probability of coming from S_0 = 0.0
Probability of coming from S_1 = 1.0

4.975234681986382
Probability of coming from S_0 = 0.0
Probability of coming from S_1 = 1.0

5.104608095902072
Probability of coming from S_0 = 7.896654698864794e-68
Probability of coming from S_1 = 1.0

5.223705980092602
Probability of coming from S_0 = 0.0
Probability of coming from S_1 = 1.0

6.468840475968554
Probability of coming from S_0 = 0.0
Probability of coming from S_1 = 1.0

5.1676174875855985
Probability of coming from S_0 = 3.1812289313800817e-32
Probability of coming from S_1 = 1.0

5.09225063745143
Probability of coming from S_0 = 2.2185454336477307e-122
Probability of coming from S_1 = 1.0

5.134435613662363
Probability of coming from S_0 = 0.6676909779738132
Probability of coming from S_1 = 0.33230902202618684

5.3991566008498735
Probability of coming from S_0 = 0.0
Probability of coming from S_1 = 1.0

8.246042743508399
Probability of coming from S_0 = 0.0
Probability of comi

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [8]:
hard_assignments = []  # List to store hard cluster assignments

for p_s0, p_s1 in zip(prob_s0_x, prob_s1_x):
    if p_s0 > p_s1:
        hard_assignments.append(0)  # Assign to cluster 0
    else:
        hard_assignments.append(1)  # Assign to cluster 1

# Print the hard assignments
for i, assignment in enumerate(hard_assignments):
    print(f"Data point {i} belongs to Cluster {assignment}")

Data point 0 belongs to Cluster 1
Data point 1 belongs to Cluster 1
Data point 2 belongs to Cluster 1
Data point 3 belongs to Cluster 1
Data point 4 belongs to Cluster 1
Data point 5 belongs to Cluster 1
Data point 6 belongs to Cluster 1
Data point 7 belongs to Cluster 0
Data point 8 belongs to Cluster 1
Data point 9 belongs to Cluster 1
